In [20]:
import dspy
from dspy.evaluate import Evaluate
from dspy.teleprompt import BootstrapFewShot, BootstrapFewShotWithRandomSearch, BootstrapFinetune
import os
from dotenv import load_dotenv
import pathlib
import re
import pandas as pd
from sklearn.metrics import f1_score, precision_score, recall_score
import dsp
import numpy as np
from scipy import sparse
import json
import pathlib
import os

In [21]:
path_env = pathlib.Path(os.getcwd()).parent.parent / '.env'
print(path_env)
load_dotenv(path_env)
api_key = os.getenv("OPENAI_API_KEY")

os.environ["OPENAI_API_KEY"] = api_key

/export/usuarios_ml4ds/lbartolome/Repos/umd/LinQAForge/.env


In [22]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

In [23]:
os.environ["DSP_NOTEBOOK_CACHEDIR"] = os.path.join('.', 'cache')

In [24]:
llm = dspy.OpenAI(
    model="gpt-4o-2024-05-13", #"gpt-4-0125-preview",  #gpt-4o-2024-05-13, #"gpt-4-1106-preview", # TODO: try turbo-instruct,
    max_tokens=1000
)

# Assumes the Weaviate collection has a text key `content`
dspy.settings.configure(lm=llm)

In [25]:
# Read parquets with topic information
out_es = pathlib.Path("/export/usuarios_ml4ds/lbartolome/Repos/umd/LinQAForge/data/models/POLI_FILTERED_AL/rosie_1_20/df_graph_es.parquet")
out_en = pathlib.Path("/export/usuarios_ml4ds/lbartolome/Repos/umd/LinQAForge/data/models/POLI_FILTERED_AL/rosie_1_20/df_graph_en.parquet")

df_es = pd.read_parquet(out_es)
df_en = pd.read_parquet(out_en)

In [26]:
df_en.label.unique()

array(['Health Data Analysis and Population Study',
       'Cardiovascular Health Research and Prevention',
       'Medical Treatment and Technology Evaluation',
       'Dental Hygiene and Child Health', 'Nutrition and Health Analysis',
       'Common Symptoms and Treatment of Infections',
       'Child Mental Health and Family Support',
       'Genetic Factors in Disease Development',
       'Muscle and Bone Health Study',
       'Biological Processes and Genetics Study',
       'Pediatric Health Care Services',
       'Vaccination and Disease Prevention Study',
       'Clinical Trial Analysis and Treatment Evaluation',
       'Medical Transplant Technology and Treatment Analysis',
       'Tuberculosis Outbreak Analysis',
       'Maternal Health and Pregnancy Technology',
       'Public Health Initiative and Community Support Program',
       'Healthcare Provider Consultation and Treatment Services',
       'Cancer Treatment and Imaging Technology',
       'Cardiovascular Surgery and 

In [27]:
df_en[df_en.main_topic==16]

,id_preproc,lemmas,lemmas_tr,doc_id,text,text_tr,lang,len,thetas,top_k,main_topic,label
42,42,proper hand_hygiene key stop spread stomach_bug remind family important time wash_soap bathroom change eat handle food,higiene adecuado mano clave detener_propagación insecto_estomacal recuerde familia importante lavar él agua baño cambiar comer manipular alimento,EN_537502_88517-5,Proper hand hygiene is key to stopping the spread of stomach bugs. Remind everyone in your family about the two most important times to wash up with soap and water:\n- After using the bathroom or changing diapers\n- Before eating or handling food.,La higiene adecuada de las manos es clave para detener la propagación de insectos estomacales. Recuerde a todos en su familia los dos momentos más importantes para lavarse con agua y jabón:\n- Después de usar el baño o cambiar pañales\n- Antes de comer o manipular los alimentos.,EN,16,"[(6, 0.03921568766236305), (12, 0.019607843831181526), (13, 0.37254902720451355), (16, 0.5490196347236633), (19, 0.019607843831181526)]","[[16.0, 0.5490196347236633], [13.0, 0.37254902720451355]]",16,Maternal Health and Pregnancy Technology
63,63,healthy people include public health infrastructure objective phi increase percentage population base health objective national datum available socioeconomic_status public heath surveillance datum lack information ses individual person information technology difficult describe monitor health disparity base ses residential address datum available information technology possible geocode_address link information technology census_tract ses datum conduct analysis eip recognize potential geocode_address examine possible ses disparity broad interstate level form health equity workgroup explore standardize eip state analysis demonstrate multisite analysis census_tract ses national level datum relevant prevention effort,healthy_people infraestructura salud público aumentar porcentaje salud población disponer_dato nacional socioeconómico dato vigilancia salud público carecer ees persona dificultar seguimiento disparidad_materia salud ees dato dirección_residencial disponible geocodificar_dirección vincular él dato ses tracto_censal análisis eip reconocer potencial dirección_geocodificada disparidad relacionado see amplio nivel interestatal formar grupo equidad_sanitario explorar normalización eip análisis demostrar análisis multisite ses tracto_censal proporcionar dato nivel nacional relevante actividad prevención,EN_323075_54332-16,"Healthy People 2020 includes a public health infrastructure objective (PHI 7.3) to increase the percentage of population-based health objectives for which national data are available by socioeconomic status (10). Public heath surveillance data often lack information on the SES of individual persons, making it difficult to describe and monitor health disparities based on SES. When residential address data are available, it is possible to geocode the address, link it to census tract SES data and conduct analyses. The EIP, recognizing the potential to use geocoded addresses to examine possible SES-related disparities at a broad interstate level, formed a health equity workgroup to explore standardizing its use in all EIP states (11). This analysis demonstrates that multisite analyses using census tract SES can provide national-level data that are relevant to prevention efforts.","Healthy People 2020 incluye un objetivo de infraestructura de salud pública (7.3) para aumentar el porcentaje de objetivos de salud basados en la población para los que se dispone de datos nacionales según el estado socioeconómico (10). Los datos de vigilancia de la salud pública a menudo carecen de información sobre el EES de las personas, lo que dificulta la descripción y el seguimiento de las disparidades en materia de salud basadas en el EES. Cuando los datos de la dirección residencial están disponibles, es posible geocodificar la dirección, vincularla a los datos del SES del tracto censal y 

In [28]:
example = df_en[df_en.main_topic==16].iloc[0].text
example

'Proper hand hygiene is key to stopping the spread of stomach bugs. Remind everyone in your family about the two most important times to wash up with soap and water:\\n- After using the bathroom or changing diapers\\n- Before eating or handling food.'

In [29]:
class QAGenerator(dspy.Signature):
    """Generate a question from the given context."""
    
    context = dspy.InputField(desc="A passage of text from which to generate a question.")
    question = dspy.OutputField(desc="A single, well-formed question generated from the given sentence.")

class QAAnswererEN(dspy.Signature):
    """Answer the question based only the given context in English."""
    
    context = dspy.InputField()
    question = dspy.InputField()
    answer = dspy.OutputField()
    
class QAAnswererES(dspy.Signature):
    """Answer the question based only the given context in Spanish."""
    
    context = dspy.InputField()
    question = dspy.InputField()
    answer = dspy.OutputField()


class QATranslator(dspy.Signature):
    """Translate the given context to Spanish"""
    
    question = dspy.InputField()
    tr_question = dspy.OutputField()
    

class QAChecker(dspy.Signature):
    """Identify if there is a contradiction between the answers given the question. """
    
    question = dspy.InputField()
    answer1 = dspy.InputField()
    answer2 = dspy.InputField()
    Faithfulness = dspy.OutputField(desc="True/False indicating if answer1 contradicts answer2")
    rationale = dspy.OutputField(desc="Why answer1 contradicts answer2 or not")
    
class CheckContradictionType(dspy.Signature):
    """Identify the type of contradiction detected"""
    
    question = dspy.InputField()
    answer1 = dspy.InputField()
    answer2 = dspy.InputField()
    rationale = dspy.OutputField(desc="Why the answer2 contradicts answer 1")
    LabelInt = dspy.OutputField(desc="A rating between 1 and 4: 1 = Direct Contradiction, 2 = Major Discrepancy, 3 = Minor Discrepancy, 4 = Fail to address the question")

In [ ]:
class QAPair(dspy.Module):
    def __init__(self):
        super().__init__()
        self.generator = dspy.ChainOfThought(QAGenerator)
        self.answerer = dspy.ChainOfThought(QAAnswererEN)
        self.translator = dspy.ChainOfThought(QATranslator)
    
    def forward(self, context):
        try:
            question = self.generator(context=context)
        except Exception as e:
            print(f"-- -- Error generating question: {e}")
            question = None
        
        answer = None
        if question:
            try:
                answer = self.answerer(context=context, question=question.question)
            except Exception as e:
                print(f"-- -- Error generating answer: {e}")
                answer = None
        
        tr_answer = None
        if answer:
            try:
                tr_answer = self.translator(question=question.question)
            except Exception as e:
                print(f"-- -- Error generating translation: {e}")
                tr_answer = None
        
        return dspy.Prediction(
            question=question.question if question else None,
            answer=answer.answer if answer else None,
            tr_question=tr_answer.tr_question if tr_answer else None
        )

qa_pair = QAPair()
ex_question = qa_pair(context=example)
print(ex_question)

### Optimize question generation

In [31]:
class QAPair(dspy.Module):
    def __init__(self):
        super().__init__()
        self.generator = dspy.ChainOfThought(QAGenerator)
        self.answerer = dspy.ChainOfThought(QAAnswererEN)
        self.translator = dspy.ChainOfThought(QATranslator)
    
    def forward(self, context):
        try:
            question = self.generator(context=context)
        except Exception as e:
            print(f"-- -- Error generating question: {e}")
            question = None
        
        answer = None
        if question:
            try:
                answer = self.answerer(context=context, question=question.question)
            except Exception as e:
                print(f"-- -- Error generating answer: {e}")
                answer = None
        
        return dspy.Prediction(
            question=question.question if question else None,
            answer=answer.answer if answer else None,
        )

In [139]:
example = df_en[df_en.main_topic==16].loc[879164].text
qa_pair(context=example)

Prediction(
    question='What are some common causes of physiologic atrophy and which groups of people are most affected by it?',
    answer='Some common causes of physiologic atrophy are not using the muscles enough, having seated jobs, health problems that limit movement, decreased activity levels, being bedridden, inability to move limbs due to stroke or other brain diseases, and being in a place that lacks gravity, such as during space flights. The groups of people most affected by it are those with seated jobs, health problems that limit movement, decreased activity levels, bedridden individuals, those who cannot move their limbs because of stroke or other brain diseases, and individuals in places that lack gravity, such as during space flights.',
    tr_question='¿Cuáles son algunas causas comunes de la atrofia fisiológica y qué grupos de personas son los más afectados por ella?\n\n---'
)

In [50]:
file_path = "/export/usuarios_ml4ds/lbartolome/Repos/umd/LinQAForge/dev-context-en-question-en.json"
with open(file_path, 'r') as file:
    data = json.load(file)

# Example: Flatten the JSON structure to create a DataFrame
# This example assumes you want to create a DataFrame from the 'data' field

data_flattened = []
for item in data['data']:
    title = item['title']
    for paragraph in item['paragraphs']:
        context = paragraph['context']
        for qa in paragraph['qas']:
            qa_item = {
                'title': title,
                'context': context,
                'question': qa['question'],
                #'is_impossible': qa['is_impossible'],
                'answers': qa['answers']
            }
            data_flattened.append(qa_item)

# Convert the flattened list to a DataFrame
samples_train = pd.DataFrame(data_flattened)

# Display the DataFrame
# [samples_train.is_impossible==True]
samples_train

,title,context,question,answers
0,Pappataci fever,"Pappataci fever is prevalent in the subtropical zone of the Eastern Hemisphere between 20°N and 45°N, particularly in Southern Europe, North Africa, the Balkans, Eastern Mediterranean, Iraq, Iran, Pakistan, Afghanistan and India.The disease is transmitted by the bites of phlebotomine sandflies of the Genus Phlebotomus, in particular, Phlebotomus papatasi, Phlebotomus perniciosus and Phlebotomus perfiliewi. The sandfly becomes infected when biting an infected human in the period between 48 hours before the onset of fever and 24 hours after the end of the fever, and remains infected for its lifetime. Besides this horizontal virus transmission from man to sandfly, the virus can be transmitted in insects transovarially, from an infected female sandfly to its offspring.Pappataci fever is seldom recognised in endemic populations because it is mixed with other febrile illnesses of childhood, but it is more well-known among immigrants and military personnel from non-endemic regions.",Does an infection for Sandflies go away over time?,"[{'text': 'remains infected for its lifetime', 'answer_start': 571}]"
1,Plant cell,"Collenchyma cells – collenchyma cells are alive at maturity and have thickened cellulosic cell walls. These cells mature from meristem derivatives that initially resemble parenchyma, but differences quickly become apparent. Plastids do not develop, and the secretory apparatus (ER and Golgi) proliferates to secrete additional primary wall. The wall is most commonly thickest at the corners, where three or more cells come in contact, and thinnest where only two cells come in contact, though other arrangements of the wall thickening are possible. Pectin and hemicellulose are the dominant constituents of collenchyma cell walls of dicotyledon angiosperms, which may contain as little as 20% of cellulose in Petasites. Collenchyma cells are typically quite elongated, and may divide transversely to give a septate appearance. The role of this cell type is to support the plant in axes still growing in length, and to confer flexibility and tensile strength on tissues. The primary wall lacks lignin that would make it tough and rigid, so this cell type provides what could be called plastic support – support that can hold a young stem or petiole into the air, but in cells that can be stretched as the cells around them elongate. Stretchable support (without elastic snap-back) is a good way to describe what collenchyma does. Parts of the strings in celery are collenchyma.",What is the name of a type of string that constitutes parts of celery?,"[{'text': 'collenchyma', 'answer_start': 1364}]"
2,Plant cell,"Parenchyma cells are living cells that have functions ranging from storage and support to photosynthesis (mesophyll cells) and phloem loading (transfer cells). Apart from the xylem and phloem in their vascular bundles, leaves are composed mainly of parenchyma cells. Some parenchyma cells, as in the epidermis, are specialized for light penetration and focusing or regulation of gas exchange, but others are among the least specialized cells in plant tissue, and may remain totipotent, capable of dividing to produce new populations of undifferentiated cells, throughout their lives. Parenchyma cells have thin, permeable primary walls enabling the transport of small molecules between them, and their cytoplasm is responsible for a wide range of biochemical functions such as nectar secretion, or the manufacture of secondary products that discourage herbivory. Parenchyma cells that contain many chloroplasts and are concerned primarily with photosynthesis are called chlorenchyma cells. Others, such as the majority of the parenchyma cells in potato tubers and the seed cotyledons of legumes, have a storage function.",What are made up almost entirely of parenchyma cells?,"[{'text': 'leaves', 'answer_start': 220}]"
3,Plant cell,"Phloem is a specialised tissue for food transport in higher plants, mainly 

In [56]:
file_path = "/export/usuarios_ml4ds/lbartolome/Repos/umd/LinQAForge/dev-context-en-question-es.json"
with open(file_path, 'r') as file:
    data = json.load(file)

# Example: Flatten the JSON structure to create a DataFrame
# This example assumes you want to create a DataFrame from the 'data' field

data_flattened = []
for item in data['data']:
    title = item['title']
    for paragraph in item['paragraphs']:
        context = paragraph['context']
        for qa in paragraph['qas']:
            qa_item = {
                'title': title,
                'context': context,
                'question': qa['question'],
                #'is_impossible': qa['is_impossible'],
                'answers': qa['answers']
            }
            data_flattened.append(qa_item)

# Convert the flattened list to a DataFrame
samples_train_es = pd.DataFrame(data_flattened)

# Display the DataFrame
# [samples_train.is_impossible==True]
samples_train_es

,title,context,question,answers
0,Pappataci fever,"Pappataci fever is prevalent in the subtropical zone of the Eastern Hemisphere between 20°N and 45°N, particularly in Southern Europe, North Africa, the Balkans, Eastern Mediterranean, Iraq, Iran, Pakistan, Afghanistan and India.The disease is transmitted by the bites of phlebotomine sandflies of the Genus Phlebotomus, in particular, Phlebotomus papatasi, Phlebotomus perniciosus and Phlebotomus perfiliewi. The sandfly becomes infected when biting an infected human in the period between 48 hours before the onset of fever and 24 hours after the end of the fever, and remains infected for its lifetime. Besides this horizontal virus transmission from man to sandfly, the virus can be transmitted in insects transovarially, from an infected female sandfly to its offspring.Pappataci fever is seldom recognised in endemic populations because it is mixed with other febrile illnesses of childhood, but it is more well-known among immigrants and military personnel from non-endemic regions.",Una infección de moscas de la arena ¿desaparece con el tiempo?,"[{'text': 'remains infected for its lifetime', 'answer_start': 571}]"
1,Plant cell,"Collenchyma cells – collenchyma cells are alive at maturity and have thickened cellulosic cell walls. These cells mature from meristem derivatives that initially resemble parenchyma, but differences quickly become apparent. Plastids do not develop, and the secretory apparatus (ER and Golgi) proliferates to secrete additional primary wall. The wall is most commonly thickest at the corners, where three or more cells come in contact, and thinnest where only two cells come in contact, though other arrangements of the wall thickening are possible. Pectin and hemicellulose are the dominant constituents of collenchyma cell walls of dicotyledon angiosperms, which may contain as little as 20% of cellulose in Petasites. Collenchyma cells are typically quite elongated, and may divide transversely to give a septate appearance. The role of this cell type is to support the plant in axes still growing in length, and to confer flexibility and tensile strength on tissues. The primary wall lacks lignin that would make it tough and rigid, so this cell type provides what could be called plastic support – support that can hold a young stem or petiole into the air, but in cells that can be stretched as the cells around them elongate. Stretchable support (without elastic snap-back) is a good way to describe what collenchyma does. Parts of the strings in celery are collenchyma.",¿Cómo se llama un tipo de hilo que forma parte del apio?,"[{'text': 'collenchyma', 'answer_start': 1364}]"
2,Plant cell,"Parenchyma cells are living cells that have functions ranging from storage and support to photosynthesis (mesophyll cells) and phloem loading (transfer cells). Apart from the xylem and phloem in their vascular bundles, leaves are composed mainly of parenchyma cells. Some parenchyma cells, as in the epidermis, are specialized for light penetration and focusing or regulation of gas exchange, but others are among the least specialized cells in plant tissue, and may remain totipotent, capable of dividing to produce new populations of undifferentiated cells, throughout their lives. Parenchyma cells have thin, permeable primary walls enabling the transport of small molecules between them, and their cytoplasm is responsible for a wide range of biochemical functions such as nectar secretion, or the manufacture of secondary products that discourage herbivory. Parenchyma cells that contain many chloroplasts and are concerned primarily with photosynthesis are called chlorenchyma cells. Others, such as the majority of the parenchyma cells in potato tubers and the seed cotyledons of legumes, have a storage function.",¿Qué se compone casi por completo de células de parénquima?,"[{'text': 'leaves', 'answer_start': 220}]"
3,Plant cell,"Phloem is a specialised tissue for food transport in higher plants, mai

In [58]:
#samples_train_es.answers.values

In [46]:
samples_train["answers"] = samples_train["answers"].apply(lambda x: "I can't answer that question given the context" if x == [] else x)

In [48]:
samples_train[samples_train.is_impossible==True]

,title,context,question,is_impossible,answers
2075,The_Legend_of_Zelda:_Twilight_Princess,"The Legend of Zelda: Twilight Princess (Japanese: ゼルダの伝説 トワイライトプリンセス, Hepburn: Zeruda no Densetsu: Towairaito Purinsesu?) is an action-adventure game developed and published by Nintendo for the GameCube and Wii home video game consoles. It is the thirteenth installment in the The Legend of Zelda series. Originally planned for release on the GameCube in November 2005, Twilight Princess was delayed by Nintendo to allow its developers to refine the game, add more content, and port it to the Wii. The Wii version was released alongside the console in North America in November 2006, and in Japan, Europe, and Australia the following month. The GameCube version was released worldwide in December 2006.[b]",What category of game is Legend of Zelda: Australia Twilight?,True,I can't answer that question given the context
2076,The_Legend_of_Zelda:_Twilight_Princess,"The Legend of Zelda: Twilight Princess (Japanese: ゼルダの伝説 トワイライトプリンセス, Hepburn: Zeruda no Densetsu: Towairaito Purinsesu?) is an action-adventure game developed and published by Nintendo for the GameCube and Wii home video game consoles. It is the thirteenth installment in the The Legend of Zelda series. Originally planned for release on the GameCube in November 2005, Twilight Princess was delayed by Nintendo to allow its developers to refine the game, add more content, and port it to the Wii. The Wii version was released alongside the console in North America in November 2006, and in Japan, Europe, and Australia the following month. The GameCube version was released worldwide in December 2006.[b]",What consoles can be used to play Australia Twilight?,True,I can't answer that question given the context
2077,The_Legend_of_Zelda:_Twilight_Princess,"The Legend of Zelda: Twilight Princess (Japanese: ゼルダの伝説 トワイライトプリンセス, Hepburn: Zeruda no Densetsu: Towairaito Purinsesu?) is an action-adventure game developed and published by Nintendo for the GameCube and Wii home video game consoles. It is the thirteenth installment in the The Legend of Zelda series. Originally planned for release on the GameCube in November 2005, Twilight Princess was delayed by Nintendo to allow its developers to refine the game, add more content, and port it to the Wii. The Wii version was released alongside the console in North America in November 2006, and in Japan, Europe, and Australia the following month. The GameCube version was released worldwide in December 2006.[b]",When was Australia Twilight launched in North America?,True,I can't answer that question given the context
2078,The_Legend_of_Zelda:_Twilight_Princess,"The Legend of Zelda: Twilight Princess (Japanese: ゼルダの伝説 トワイライトプリンセス, Hepburn: Zeruda no Densetsu: Towairaito Purinsesu?) is an action-adventure game developed and published by Nintendo for the GameCube and Wii home video game consoles. It is the thirteenth installment in the The Legend of Zelda series. Originally planned for release on the GameCube in November 2005, Twilight Princess was delayed by Nintendo to allow its developers to refine the game, add more content, and port it to the Wii. The Wii version was released alongside the console in North America in November 2006, and in Japan, Europe, and Australia the following month. The GameCube version was released worldwide in December 2006.[b]",When could GameCube owners purchase Australian Princess?,True,I can't answer that question given the context
2079,The_Legend_of_Zelda:_Twilight_Princess,"The Legend of Zelda: Twilight Princess (Japanese: ゼルダの伝説 トワイライトプリンセス, Hepburn: Zeruda no Densetsu: Towairaito Purinsesu?) is an action-adventure game developed and published by Nintendo for the GameCube and Wii home video game consoles. It is the thirteenth installment in the The Legend of Zelda series. Originally planned for release on the GameCube in November 2005, Twilight Princess was delayed by Nintendo to allow its developers to refine the game, add more 

In [158]:
class PipelineChecker(dspy.Module):
    def __init__(self, index_en, model_en, ids_en, texts_en, index_es, model_es, ids_es, texts_es):
        super().__init__()
        self.answerer_en = dspy.ChainOfThought(QAAnswererEN)
        self.answerer_es = dspy.ChainOfThought(QAAnswererES)
        self.checker = dspy.ChainOfThought(QAChecker)
        self.type_checker = dspy.ChainOfThought(CheckContradictionType)
        self.index_en = index_en
        self.model_en = model_en
        self.ids_en = ids_en
        self.texts_en = texts_en
        self.index_es = index_es
        self.model_es = model_es
        self.ids_es = ids_es
        self.texts_es = texts_es
    
    def process_response(self, checker_response):
        if "true" in checker_response.Faithfulness.lower():
            return True
        elif "false" in checker_response.Faithfulness.lower():
            return False
        else:
            print("-- --- Non-valid response from Checker")
            
    def process_contr_type(self, contr_type_response):
        if "1" in contr_type_response.LabelInt:
            return 1
        elif "2" in contr_type_response.LabelInt:
            return 2
        elif "3" in contr_type_response.LabelInt:
            return 3
        elif "4" in contr_type_response.LabelInt:
            return 4
    
    def forward(self, question_en, question_es, true_answer, k = 5):
        # Retrieve k similar documents from the English corpus
        results_en = retrieve_similar_documents(question_en, self.model_en, self.index_en, self.ids_en, self.texts_en, k)
        context_en = [result['text'] for result in results_en]
        
        # Retrieve k similar documents from the Spanish corpus
        results_es = retrieve_similar_documents(question_es, self.model_es, self.index_es, self.ids_es, self.texts_es, k)
        context_es = [result['text'] for result in results_es]
        
        # Answer the question in English
        answer_en = self.answerer_en(context=' '.join(context_en), question=question_en).answer

        # Answer the question in Spanish
        answer_es = self.answerer_es(context=' '.join(context_es), question=question_es).answer

        # Check if the answer in English aligns with the true answer
        #check_en = self.checker(question=question_en, answer1=true_answer, answer2=answer_en)

        # Check if the answer in Spanish aligns with the answer in English
        check_alignment = self.checker(question=question_en, answer1=answer_en, answer2=answer_es)
        
        print(check_alignment)
        
        if self.process_response(check_alignment):
            contr_type = self.type_checker(question=question_en, answer1=answer_en, answer2=answer_es)
            contr_type_label = self.process_contr_type(contr_type)
            contr_type_rationale = contr_type.rationale
        else:
            contr_type_label = None
            contr_type_rationale = None
        

        return {
            "question": question_en,
            "answer_en": answer_en,
            "answer_es": answer_es,
            #"check_en_en": self.process_response(check_en),
            "check_en_es": self.process_response(check_alignment),
            #"check_en_en_rat": check_en.rationale,
            "check_en_es_rat": check_alignment.rationale,
            "contr_type_label": contr_type_label,
            "contr_type_rationale": contr_type_rationale
        }

## FAISS index

In [17]:
df_index_es = df_es[df_es.main_topic==16][['id_preproc', 'text']]
df_index_en = df_en[df_en.main_topic==16][['id_preproc', 'text']]

In [ ]:
def create_faiss_index(df, text_column, id_column, model_name="all-mpnet-base-v2", index_file="faiss_index.index"):
    """
    Create a FAISS index from a DataFrame containing text data.

    Parameters:
    df (pd.DataFrame): The input DataFrame.
    text_column (str): The name of the column containing text data.
    id_column (str): The name of the column containing unique identifiers for the texts.
    model_name (str): The name of the SentenceTransformer model to use for embeddings.
    index_file (str): The file path to save the FAISS index.

    Returns:
    index: The FAISS index object.
    model: The SentenceTransformer model used for embeddings.
    ids: List of document IDs.
    texts: List of document texts.
    """
    # Extract texts and ids
    texts = df[text_column].tolist()
    ids = df[id_column].tolist()

    # Load pre-trained SentenceTransformer model
    model = SentenceTransformer(model_name, device="cuda")

    # Calculate embeddings for the texts
    embeddings = model.encode(texts, show_progress_bar=False)

    # Create a FAISS index
    d = embeddings.shape[1]
    index = faiss.IndexFlatIP(d)  # Using Inner Product to approximate cosine similarity

    # Normalize the embeddings to unit length for cosine similarity
    faiss.normalize_L2(embeddings)

    # Add embeddings to the index
    index.add(embeddings)

    # Save the index to a file
    faiss.write_index(index, index_file)

    return index, model, ids, texts

def retrieve_similar_documents(query_text, model, index, ids, texts, k=5):
    """
    Retrieve the k most similar documents to the query text.

    Parameters:
    query_text (str): The query text.
    model: The SentenceTransformer model used for embeddings.
    index: The FAISS index object.
    ids (list): List of document IDs.
    texts (list): List of document texts.
    k (int): The number of nearest neighbors to retrieve.

    Returns:
    list: A list of dictionaries containing document IDs, distances, and texts of the k most similar documents.
    """
    # Encode the query text
    query_embedding = model.encode([query_text], show_progress_bar=False)
    faiss.normalize_L2(query_embedding)
    
    # Search the index for the k nearest neighbors
    distances, indices = index.search(query_embedding, k)
    
    # Retrieve the corresponding texts and ids
    results = []
    for i in range(k):
        result = {
            "document_id": ids[indices[0][i]],
            "distance": distances[0][i],
            "text": texts[indices[0][i]]
        }
        results.append(result)
    
    return results


index_es, model_es, ids_es, texts_es = create_faiss_index(df_index_es, text_column='text', id_column='id_preproc', index_file='faiss_index_es.index')
index_en, model_en, ids_en, texts_en = create_faiss_index(df_index_en, text_column='text', id_column='id_preproc', index_file='faiss_index_en.index')

# Example query
query_text = "Proper hand hygiene is key to stopping the spread of diseases."
results = retrieve_similar_documents(query_text, model_en, index_en, ids_en, texts_en)

In [80]:
pipeline_checker = PipelineChecker(index_en, model_en, ids_en, texts_en, index_es, model_es, ids_es, texts_es)
result = pipeline_checker.forward(ex_question.question, ex_question.tr_question, ex_question.answer)
print(result)

{'question': 'What are the two most important times to wash your hands with soap and water to prevent the spread of stomach bugs?', 'answer_en': 'Context: Proper hand hygiene is key to stopping the spread of stomach bugs. Remind everyone in your family about the two most important times to wash up with soap and water:\n- After using the bathroom or changing diapers\n- Before eating or handling food. Washing Hands: The single most important means of preventing the spread of infection. It is best to wash with soap and water', 'answer_es': '---\n\nContext: Preparación para colocar la sonda: 1. Lávese las manos con agua y jabón. Beba abundante agua antes, durante y después de hacer actividad física. Esto es muy importante cuando haga mucho calor y cuando haga ejercicio muy intenso. En las escuelas: Las escuelas usan una cantidad formidable de agua todos los días y requieren agua para los sistemas', 'check_en_es': True, 'check_en_es_rat': 'Answer 1 provides specific times to wash hands to p

## Demo on Topic 16

In [ ]:
qa_pair = QAPair()
pipeline_checker = PipelineChecker(index_en, model_en, ids_en, texts_en, index_es, model_es, ids_es, texts_es)

checked = []
for index, row in df_index_en.sample(n=20, random_state=743_34).iterrows():
    # Generate question
    ex_question = qa_pair(context=row.text)
    
    # Perform the QA pair checking
    result = pipeline_checker.forward(ex_question.question, ex_question.tr_question, ex_question.answer)
    
    checked.append({
        "question": ex_question.question,
        "tr_question": ex_question.tr_question,
        "orig_answer": ex_question.answer,
        **result
    })

# Create a DataFrame from the checked list
checked_df = pd.DataFrame(checked)

- Remove too specific questions
- Spanish corpus may be too small (at the script level during the construction of the initial dataset)

In [18]:
checked_df.question.unique()

NameError: name 'checked_df' is not defined

In [162]:
checked_df

,question,tr_question,orig_answer,answer_en,answer_es,check_en_es,check_en_es_rat,contr_type_label,contr_type_rationale
0,What are some foods that contain ginger and may help ease nausea for moms-to-be?,¿Cuáles son algunos alimentos que contienen jengibre y pueden ayudar a aliviar las náuseas en las futuras mamás?\n\n---,"Ginger ale, ginger lollipops, and ginger tea.","Ginger ale, ginger lollipops, and ginger tea.",El contexto no proporciona información sobre alimentos que contienen jengibre ni sobre cómo aliviar las náuseas en las futuras mamás.\n\n---,True,"Answer 1 provides specific examples of foods and drinks that contain ginger and may help ease nausea for moms-to-be, such as ginger ale, ginger lollipops, and ginger tea. Answer 2, however, states that the context does not provide information about foods containing ginger or how to alleviate nausea in expectant mothers. This directly contradicts the information given in Answer 1.",1.0,"produce the LabelInt. We need to evaluate the consistency between the two answers provided.\n\n1. Answer 1 lists specific foods and beverages that contain ginger and are known to help ease nausea: ginger ale, ginger lollipops, and ginger tea.\n2. Answer 2 states that the context does not provide information about foods containing ginger or how to alleviate nausea in expectant mothers.\n\nGiven that Answer 1 provides specific examples directly related to the question, while Answer 2 claims that no such information is provided, there is a clear contradiction between the two answers."
1,What are some common causes of physiologic atrophy and which groups of people are most affected by it?,¿Cuáles son algunas causas comunes de la atrofia fisiológica y qué grupos de personas son los más afectados por ella?\n\n---,"Some common causes of physiologic atrophy include not using the muscles enough, having seated jobs, health problems that limit movement, decreased activity levels, being bedridden, inability to move limbs due to stroke or other brain diseases, and being in a place that lacks gravity, such as during space flights. The groups of people most affected by physiologic atrophy are those with seated jobs, health problems that limit movement, decreased activity levels, bedridden individuals, those who cannot move their limbs because of stroke or other brain diseases, and individuals in environments lacking gravity, such as astronauts during space flights.","Some common causes of physiologic atrophy include not using the muscles enough, lack of exercise, aging, genetic predispositions, and changes in hormones. The groups of people most affected by physiologic atrophy are those who have seated jobs, health problems that limit movement, or decreased activity levels, are bedridden, cannot move their limbs because of stroke or other brain disease, or are in a place that lacks gravity, such as during space flights.",La información proporcionada no incluye detalles sobre las causas comunes de la atrofia fisiológica ni sobre los grupos de personas más afectados por ella.,True,"Answer 1 provides specific details about the common causes of physiologic atrophy and the groups of people most affected by it. In contrast, Answer 2 explicitly states that the information provided does not include details about the causes or the affected groups. This directly contradicts the detailed information given in Answer 1.",1.0,"Reasoning: Let's think step by step in order to produce the LabelInt. We need to evaluate the consistency and relevance of the answers provided to the question asked.\n\n1. The question asks for common causes of physiologic atrophy and the groups of people most affected by it.\n2. Answer 1 provides a detailed list of causes (e.g., lack of exercise, aging, genetic predispositions, changes in hormones) and identifies specific groups of people affected (e.g., those with seated jobs, health problems, bedridden individuals, stroke patients, astronauts).\n3. Answer 2 states that the provided information

## Reading some answers

In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util

df = pd.read_csv("/export/usuarios_ml4ds/lbartolome/Repos/umd/LinQAForge/data/500EnglishSample.csv")
df2 = pd.read_csv("/export/usuarios_ml4ds/lbartolome/Repos/umd/LinQAForge/data/500SpanishSample.csv")

# Load multilingual sentence transformer model
model = SentenceTransformer('distiluse-base-multilingual-cased-v2')

# Get question columns
questions_df1 = df['question'].tolist()
answers_df1 = df['answerPassageText'].tolist()
questions_df2 = df2['question'].tolist()
answers_df2 = df2['answerPassageText'].tolist()

# Compute embeddings for each set of questions
embeddings_df1 = model.encode(questions_df1, convert_to_tensor=True)
embeddings_df2 = model.encode(questions_df2, convert_to_tensor=True)

# Find pairs of questions with high similarity
cosine_scores = util.pytorch_cos_sim(embeddings_df1, embeddings_df2)

In [12]:
# Set a threshold for considering questions as similar (e.g., 0.8)
threshold = 0.8
common_questions = []

for i in range(len(questions_df1)):
    for j in range(len(questions_df2)):
        if cosine_scores[i][j] > threshold:
            common_questions.append({
                'Question in DF1': questions_df1[i],
                'Answer in DF1': answers_df1[i],
                'Question in DF2': questions_df2[j],
                'Answer in DF2': answers_df2[j]
            })

# Create a dataframe for common questions and answers
common_df = pd.DataFrame(common_questions)

# Filter out the empty rows
common_df = common_df.dropna()
common_df = common_df.drop_duplicates(subset=['Question in DF1'])

In [13]:
common_df.head(20)

,Question in DF1,Answer in DF1,Question in DF2,Answer in DF2
0,What is the right way to bathe a baby,"When Should I Bathe My Baby: For the first few weeks, babies get sponge baths. Then, after the umbilical cord stump falls off and the circumcision heals (if your baby was circumcised), babies can have baths in a sink or small plastic infant tub.\n\n<a href=""https://kidshealth.org/en/parents/guide-parents.html"">Continue reading here</a>",¿Cuáles son algunas formas seguras de bañar a mi bebé?,¿Cómo puedo garantizar la seguridad de mi bebé durante los baños:<br>-\n\nContinúe leyendo aquí: https://kidshealth.org/es/parents/bathing-baby.html
3,How many hours should a newborn sleep?,"How long should a newborn sleep each day: There isn’t really a “should” for hours of sleep for a newborn, because there is such a wide range of normal. For any age, Moore recommends viewing specific numbers as general guidelines. Better indicators are that your child is happy and healthy and waking on their own in the morning when given the opportunity.\n\nContinue reading here: https://www.chop.edu/news/health-tip/baby-sleep-questions-answered",¿Cuánto tiempo debe dormir la siesta mi bebé?,"La mayoría de los bebés duermen un total de 18 horas al día. Se despiertan por poco tiempo, como mínimo, cada 2 o 3 horas. Siempre debe hacer dormir al bebé boca arriba, no sobre el abdomen. Esto reduce el riesgo del síndrome de muerte infantil súbita (SIDS, por sus siglas en inglés).\n\nsource: https://espanol.kaiserpermanente.org/es/southern-california/health-wellness/health-encyclopedia/he.abl1277"
16,How can I soothe my baby when they're experiencing teething pain?,"How do you soothe a teething baby: Give babies something they can chew on but not choke on. A firm rubber toy, a soft toothbrush, a cool ring or a cool rag are all good options. Some parents massage their own fingers along the baby’s gums, which can provide comfort. If all else fails, a dose of acetaminophen (Tylenol) can do the trick to ease pain.\n\nContinue reading here: https://www.hopkinsallchildrens.org/ACH-News/General-News/What-You-Should-Know-about-Babies-Teething",¿Cómo puedo calmar el cólico de mi bebé?,"Calmar al bebé: Después de que comienza un episodio de cólicos, puede poner en práctica medidas para calmar al bebé.\n\nContinúe leyendo aquí: https://espanol.kaiserpermanente.org/es/southern-california/health-wellness/health-encyclopedia/he.hw31504"
19,What will I need at the hospital for delivery?,"Delivery: You may be able to deliver your baby at your local hospital. If there are concerns your baby may have symptoms at birth, you will need to deliver your baby at a hospital that is ready to care for high-risk babies like yours. In rare cases, it may be recommended that you deliver at a children’s hospital because the team thinks your baby is at high risk for needing urgent surgery. You should discuss your plans for labor and delivery with your obstetrician. Contact your doctor right away if you have any concerns.\n\nContinue reading here: https://www.nationwidechildrens.org/conditions/congenital-lung-lesions",¿Qué cosas debo empacar en mi bolsa de hospital para el parto y el parto?,Elementos para el personal del hospital: Cosas que necesitará en el hospital:\n\nContinúe leyendo aquí: https://medlineplus.gov/spanish/ency/patientinstructions/000543.htm
21,What are the benefits of breast feeding for baby,"What are the benefits of breastfeeding: Breastfeeding is good for your baby and good for you. Breast milk has the right amount of fat, sugar, water, and protein needed for your baby’s growth. Breast milk contains antibodies that protect babies from infections. Breastfed babies have a lower risk of sudden infant death syndrome (SIDS).\n\nContinue reading here: https://www.acog.org/womens-health/experts-and-stories/ask-acog/what-are-the-benefits-of-breastfeeding",¿Cuáles son los beneficios de la lactancia materna tanto para la madre como para el bebé?,"¿Cuáles son los beneficios de la lac

In [17]:
import json
import requests
import logging

logging.basicConfig(level=logging.DEBUG)


header = {'Content-Type': 'application/x-www-form-urlencoded'}


# Define question data and build the request dictionary
question_data = {
        #"text": "Why my baby has Jaundice?",
        "text": "¿How much sleep does a 5 month old need?",
        "userUID": "lorena-trial",
        "lang": "en"
}

# The API endpoint to communicate with
url_post = "https://rosie.umiacs.umd.edu/ask"

# A POST request to the API
post_response = requests.post(url_post, data=question_data, headers=header)

# Print the response
post_response_json = post_response.content
print(post_response_json)


DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): rosie.umiacs.umd.edu:443
DEBUG:urllib3.connectionpool:https://rosie.umiacs.umd.edu:443 "POST /ask HTTP/1.1" 200 767


b'{"recipient_id": "rosie", "text": "How many naps a day does a baby need: It takes a while for newborns to develop a sleep schedule because they need to develop their circadian rhythms. During the first month, babies will spend around 16 hours a day sleeping. This will usually happen in the form of three- or four-hour naps evenly spaced between feedings. After a newborn has been awake for one to two hours, he or she will need to sleep again.\\n\\nContinue reading here: https://www.mayoclinic.org/healthy-lifestyle/infant-and-toddler-health/in-depth/baby-naps/art-20047421", "source_document_url": "https://www.mayoclinic.org/healthy-lifestyle/infant-and-toddler-health/in-depth/baby-naps/art-20047421", "source_document_title": "English", "predicted_intent": "ask"}'
